In [1]:
import pandas as pd
import numpy as np

In [2]:
df_train = pd.read_csv("../dataset/train_cleaned_outliers_imputed.csv")
df_train_id = df_train.pop("Id")
df_train_target = df_train.pop("CO2 Emissions(g/km)")

df_test = pd.read_csv("../dataset/test_cleaned_outliers_imputed.csv")
df_test_id = df_test.pop("Id")

In [3]:
df_train.info()

onehot_columns = ["Make", "Fuel Type", "Transmission_Type", "Vehicle Class General", "Gears", "is_outlier"]
binary_columns = ["Vehicle Class", "Transmission"]
ordinal_columns = ["Vehicle Type"]

numerical_columns = df_train.select_dtypes(include='float64').columns.tolist()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54937 entries, 0 to 54936
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Make                   54937 non-null  object 
 1   Vehicle Class          54937 non-null  object 
 2   Engine Size(L)         54937 non-null  float64
 3   Cylinders              54937 non-null  float64
 4   Transmission           54937 non-null  object 
 5   Fuel Type              54937 non-null  object 
 6   Fuel Consumption City  54937 non-null  float64
 7   Fuel Consumption Hwy   54937 non-null  float64
 8   Fuel Consumption Comb  54937 non-null  float64
 9   Transmission_Type      54937 non-null  object 
 10  Gears                  54937 non-null  int64  
 11  Vehicle Class General  54937 non-null  object 
 12  Vehicle Type           54937 non-null  object 
 13  is_outlier             54937 non-null  int64  
dtypes: float64(5), int64(2), object(7)
memory usage: 5.9+ 

In [4]:
import functions as func

X_train = df_train.copy()
y_train = df_train_target.copy()

X_train, _ = func.encode_categorical_features(X_train, df_train.copy(), onehot_columns, binary_columns, ordinal_columns)
X_train, _ = func.standard_scale_dataframe(X_train, df_train.copy(), numerical_columns)

In [18]:
from bayes_opt import BayesianOptimization
import xgboost as xgb

# Load data
data = X_train.copy()
target = y_train.copy()


# Define the function to optimize
def xgb_evaluate(
    max_depth,
    gamma,
    colsample_bytree,
    subsample,
    eta,
    min_child_weight,
    max_delta_step,
    scale_pos_weight,
    reg_alpha,
    reg_lambda,
):
    params = {
        "eval_metric": "rmse",
        "max_depth": int(max_depth),
        "subsample": subsample,
        "eta": eta,
        "gamma": gamma,
        "min_child_weight": min_child_weight,
        "colsample_bytree": colsample_bytree,
        "max_delta_step": max_delta_step,
        "scale_pos_weight": scale_pos_weight,
        "reg_alpha": reg_alpha,
        "reg_lambda": reg_lambda,
    }
    cv_result = xgb.cv(params, dtrain, num_boost_round=1000, nfold=3, stratified=False)

    return -1.0 * cv_result["test-rmse-mean"].iloc[-1]


# Convert the data into a DMatrix
dtrain = xgb.DMatrix(data, label=target)

xgb_bo = BayesianOptimization(
    xgb_evaluate,
    {
        "max_depth": (1, 10),
        "gamma": (0, 1),
        "colsample_bytree": (0.3, 0.9),
        "subsample": (0.5, 1),
        "eta": (0.01, 0.3),
        "min_child_weight": (0.01, 0.3),
        "max_delta_step": (0, 10),
        "scale_pos_weight": (1, 100),
        "reg_alpha": (0, 1),
        "reg_lambda": (0, 1),
    },
)
xgb_bo.maximize(init_points=10, n_iter=100)

|   iter    |  target   | colsam... |    eta    |   gamma   | max_de... | max_depth | min_ch... | reg_alpha | reg_la... | scale_... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------
| 1         | -20.17    | 0.4004    | 0.2146    | 0.08276   | 1.543     | 5.739     | 0.2465    | 0.5844    | 0.9947    | 6.425     | 0.617     |
| 2         | -20.73    | 0.5798    | 0.271     | 0.1119    | 9.286     | 3.218     | 0.02169   | 0.5426    | 0.2635    | 22.64     | 0.5007    |
| 3         | -19.92    | 0.3148    | 0.07119   | 0.3792    | 9.698     | 5.455     | 0.254     | 0.1083    | 0.1787    | 32.21     | 0.7864    |
| 4         | -52.96    | 0.7006    | 0.1755    | 0.8491    | 0.1203    | 6.263     | 0.1312    | 0.4626    | 0.7194    | 45.79     | 0.7093    |
| 5         | -23.31    | 0.3372    | 0.1371    | 0.6418    | 2.185     | 2.828     | 0.1256    | 0.9005    | 0.762     | 4.

In [19]:
xgb_bo.max

{'target': -19.313372947948267,
 'params': {'colsample_bytree': 0.3,
  'eta': 0.03508222219931691,
  'gamma': 0.7686173125574116,
  'max_delta_step': 0.0,
  'max_depth': 8.350757895869842,
  'min_child_weight': 0.3,
  'reg_alpha': 0.19008440126889958,
  'reg_lambda': 0.7610085486525633,
  'scale_pos_weight': 13.442385709119758,
  'subsample': 1.0}}

In [15]:
from bayes_opt import BayesianOptimization
import lightgbm as lgb
from sklearn.datasets import load_iris
from sklearn.metrics import mean_squared_error

# Load data
data = X_train.copy()
target = y_train.copy()


# Define the function to optimize
def lgbm_evaluate(
    max_depth,
    num_leaves,
    min_data_in_leaf,
    feature_fraction,
    bagging_fraction,
    learning_rate,
    lambda_l1,
    lambda_l2,
    min_gain_to_split,
):
    params = {
        "objective": "regression",
        "metric": "rmse",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "max_depth": int(max_depth),
        "num_leaves": int(num_leaves),
        "min_data_in_leaf": int(min_data_in_leaf),
        "feature_fraction": feature_fraction,
        "bagging_fraction": bagging_fraction,
        "bagging_freq": 1,
        "learning_rate": learning_rate,
        "lambda_l1": int(lambda_l1),
        "lambda_l2": int(lambda_l2),
        "min_gain_to_split": int(min_gain_to_split),
    }
    train_data = lgb.Dataset(data, label=target)
    cv_result = lgb.cv(
        params, train_data, num_boost_round=1000, nfold=3, stratified=False
    )

    return -1.0 * cv_result["valid rmse-mean"][-1]


# Define the Bayesian Optimization method
lgbm_bo = BayesianOptimization(
    lgbm_evaluate,
    {
        "max_depth": (3, 30),
        "min_data_in_leaf": (10, 100),
        "feature_fraction": (0.1, 0.9),
        "bagging_fraction": (0.1, 0.9),
        "learning_rate": (0.01, 0.3),
        "num_leaves": (20, 3000),
        "min_data_in_leaf": (200, 10000),
        "lambda_l1": (0, 100),
        "lambda_l2": (0, 100),
        "min_gain_to_split": (0, 15),
    },
)
lgbm_bo.maximize(init_points=10, n_iter=100)

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | learni... | max_depth | min_da... | min_ga... | num_le... |
-------------------------------------------------------------------------------------------------------------------------------------
| 1         | -24.61    | 0.5389    | 0.6888    | 57.81     | 70.71     | 0.1841    | 22.81     | 2.733e+03 | 9.712     | 1.213e+03 |
| 2         | -36.6     | 0.4924    | 0.6678    | 83.66     | 60.58     | 0.1444    | 6.003     | 7.256e+03 | 5.734     | 2.48e+03  |
| 3         | -23.34    | 0.4365    | 0.3532    | 15.47     | 62.8      | 0.2128    | 15.13     | 1.466e+03 | 9.092     | 1.581e+03 |
| 4         | -30.09    | 0.4757    | 0.478     | 96.37     | 27.39     | 0.09884   | 5.645     | 4.71e+03  | 9.067     | 2.832e+03 |
| 5         | -67.57    | 0.3269    | 0.4506    | 7.313     | 48.84     | 0.1122    | 19.32     | 7.941e+03 | 6.385     | 2.612e+03 |
| 6         | -25.41    | 0.2569    | 0.7695    | 75.84     | 

In [16]:
lgbm_bo.max

{'target': -20.590819755716307,
 'params': {'bagging_fraction': 0.8690077395539522,
  'feature_fraction': 0.7100616390120457,
  'lambda_l1': 17.140478403668403,
  'lambda_l2': 43.36675440761465,
  'learning_rate': 0.06900334013323922,
  'max_depth': 5.299581326745951,
  'min_data_in_leaf': 210.02583586502402,
  'min_gain_to_split': 1.5177861953751233,
  'num_leaves': 2150.5332871592927}}